# Module 8: Concrete Security

## Latice Estimator

### LWE

#### Basics

In [ ]:
from estimator import *
from estimator.nd import NoiseDistribution, stddevf
from estimator.lwe_parameters import LWEParameters
from estimator.lwe import estimate

In [ ]:
MyParam = LWEParameters(n=256, q=1024, 
                Xs=NoiseDistribution.CenteredBinomial(3),
                Xe=NoiseDistribution.SparseTernary(256, 40, 40),
                m=256, 
                tag="MyParam",
                )
r = LWE.estimate.rough(MyParam)		# usvp & dual_hybrid only

In [ ]:
r = LWE.estimate(MyParam)		# more attacks

#### Cost Models

In [ ]:
from estimator.reduction import *

In [ ]:
r = LWE.estimate(MyParam, red_cost_model=ADPS16("classical"))

In [ ]:
# Check Core-SVP
print("usvp        :", numerical_approx(0.292*178, digits=3))
print("bdd         :", numerical_approx(0.292*178, digits=3))
print("dual        :", numerical_approx(0.292*192, digits=3))
print("dual_hybrid :", numerical_approx(0.292*177, digits=3))

In [ ]:
r = LWE.estimate(MyParam, red_cost_model=ADPS16("quantum"))

In [ ]:
r = LWE.estimate(MyParam, red_cost_model=ChaLoy21)

### SIS

In [ ]:
params = SIS.Parameters(n=113, q=2048, length_bound=512, norm=2)
SIS.lattice(params)

In [ ]:
params = SIS.Parameters(n=113, q=2048, length_bound=50, norm=oo)
SIS.lattice(params)

### NTRU

In [ ]:
params = NTRU.Parameters(n=200, q=7981, Xs=ND.UniformMod(3), Xe=ND.UniformMod(3))
NTRU.primal_usvp(params, red_shape_model="gsa")
NTRU.primal_usvp(params, red_shape_model=Simulator.CN11)

In [ ]:
NTRU.primal_hybrid(params, red_shape_model=Simulator.CN11)

In [ ]:
params.possibly_overstretched
NTRU.primal_dsd(params, red_shape_model=Simulator.ZGSA) 

In [ ]:
NTRU.primal_dsd(params, red_shape_model=Simulator.CN11)

## Kyber

### Parameters

In [ ]:
n = 256
q = 3329

In [ ]:
# Kyber512
tag = "Kyber512"
k = 2
eta1 = 3
eta2 = 3
eta3 = 2 # ctxt error distribution for level 1, otherwise eta2=eta3
d1 = 10
d2 = 4

In [ ]:
# Kyber768
tag = "Kyber768"
k = 3
eta1 = 2
eta2 = 2
eta3 = 2
d1 = 10
d2 = 4

In [ ]:
# Kyber1024
tag = "Kyber1024"
k = 4
eta1 = 2
eta2 = 2
eta3 = 2
d1 = 11
d2 = 5 

### Security Estimation

In [ ]:
Kyber = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.CenteredBinomial(eta1),
                	Xe=NoiseDistribution.CenteredBinomial(eta2),
                	m=n*(k+1), tag=tag)

In [ ]:
print("*"+tag+"*")
r = LWE.estimate(Kyber, red_cost_model=ADPS16("classical"))

### Decryption Failures

In [ ]:
from crystals import *
import operator as op
from math import factorial as fac
from math import sqrt, log
import sys
from crystals.proba_util import *

In [ ]:
# MLWE secrets in pk (s) and cttxt (r)   : CBD(eta1)
# MLWE errors in pk (e) and ctxt (e1, e2): CBD(eta2), CBD(eta2), CBD(eta3)
# Initialize secrets and errors
Ds = build_centered_binomial_law(eta1)      	# equal to Dr
De = build_centered_binomial_law(eta2)      	# equal to De1, De2
De1 = build_centered_binomial_law(eta3)     	# equal to De2

# Compression errors in ctxt (c1, c2), 
# each are ModSwitch errors from q to 2^d1 and 2^d2
Dc1 = build_mod_switching_error_law(q, 2**d1)
Dc2 = build_mod_switching_error_law(q, 2**d2)

# Combinations
# e*r + e2 + c2 - s*e1 - s*c1
# We ignore the publlic key compression noise, which is small
Dc1_e1 = law_convolution(Dc1, De1)                      		# c1+e1
Der = iter_law_convolution(law_product(De, Ds), n*k)        	# er
Dse1_sc1 = iter_law_convolution(law_product(Ds, Dc1_e1), n*k)  	# se1+sc1

D = law_convolution(Der, Dse1_sc1)      	# er-se1-sc1
D = law_convolution(D, De1)          		# er+e2-se1-sc1
D = law_convolution(D, Dc2)         		# final

prob = tail_probability(D, q/4)

print("*"+tag+"*")
print("DFP:", log(n*prob)/log(2))

## Smaug

### Parameters

In [ ]:
n = 256
sigma = 1.0625

In [ ]:
# Smaug512
tag = "Smaug512"
k = 2
q = 1024
p = 256
pp = 32
h = 140
numCBD = 2

In [ ]:
# Smaug768
tag = "Smaug768"
k = 3
q = 2048
p = 512
pp = 16
h = 264
numCBD = 4

In [ ]:
# Smaug512
tag = "Smaug512"
k = 4
q = 2048
p = 512
pp = 128
h = 348
numCBD = 3

### Security Estimate

Define 'ModifiedCBD' in “estimator/nd.py”:

    @staticmethod
    def ModifiedCBD(numCBD, n=None):
        #if numCBD == 1:
        #    # -1: 1/16, 0: 14/16, 1: 1/16
        if numCBD == 2:
            # -1: 1/8, 0: 3/4, 1: 1/8
            # mean = 0, stddev**2 = 1/8*1*2 = 1/4
            D = NoiseDistribution(n=n, stddev=RR(1/2), mean=RR(0), density=1/RR(4), bounds=(-1, 1), tag="modifiedCBD")
        elif numCBD == 3:
            # -1: 3/16, 0: 5/8, 1: 3/16
            # mean = 0, stddev**2 = 3/16*1*2 = 3/8
            D = NoiseDistribution(n=n, stddev=sqrt(3/RR(8)), mean=RR(0), density=3/RR(8), bounds=(-1, 1), tag="modifiedCBD")
        elif numCBD == 4:
            D = NoiseDistribution.CenteredBinomial(1)
        return D

In [ ]:
Smaug_LWE = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.SparseTernary(n*k, h/2, h/2),
                	Xe=NoiseDistribution.DiscreteGaussian(sigma),
                	m=n*k, tag=tag+"LWE")

Smaug_LWR = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.ModifiedCBD(numCBD),
                	Xe=NoiseDistribution.UniformMod(q/p),
                	m=n*(k+1), tag=tag+"LWR")

In [ ]:
print("*"+tag+"*")
r = LWE.estimate(Smaug_LWE, red_cost_model=ADPS16("classical"))

In [ ]:
print("*"+tag+"*")
r = LWE.estimate(Smaug_LWR, red_cost_model=ADPS16("classical"))

### Decryption Failures

Define 'build_mCBD_law' in “crystals/proba_util.py”:

    def build_mCBD_law(numCBD):
        D = {}
        D[-1] = numCBD/16.0
        D[1] = numCBD/16.0
        D[0] = 1.0-D[1]-D[-1]
        return D

In [ ]:
##############
# initialize #
##############
# LWE error (dGaussian, sigma=1.0625)
D_e = {-3: 7.0/1024, -2: 65.0/1024, -1: 247.0/1024, 0:384.0/1024, 1: 247.0/1024, 2: 65.0/1024, 3: 7.0/1024}

# LWR secret (modifiedCBD)
D_r = build_mCBD_law(numCBD)

# LWR error for ctxt    (ModSwitch q->p and pp)
D_e1 = build_mod_switching_error_law(q, p)
D_e2 = build_mod_switching_error_law(q, pp)

###########
# combine #
###########
D_er = iter_law_convolution(law_product(D_e, D_r), k*n)    # <e, r>
D_e1s = iter_law_convolution(D_e1, h)                     # <e1, s> = e1+...+e1, hs times.

D_er_e1s = law_convolution(D_er, D_e1s)
# convolution_remove_dependency(D_er, D_e1s, q, p)  # <e, r> + <e1, s>

D = law_convolution(D_er_e1s, D_e2)                # <e, r> + <e1, s> + e2

prob = tail_probability(D, q/4.0)                  # Pr[ |<e, r> + <e1, s> + e2| > q/4 ]

if prob!=0:
    prob = log(n*prob,2)                           # for each n coefficients

print("*"+tag+"*")
print(prob)

## Falcon

### Parameters

In [ ]:
q = 12289.0

In [ ]:
# Falcon512
tag = "Falcon512"
n = 512.0
betaSquared=34034726
Ran_KR = range(450, 460)
Ran_Unf = range(405, 415)

In [ ]:
# Falcon1024
tag = "Falcon1024"
n = 1024.0
betaSquared=70265242
Ran_KR = range(930, 940)
Ran_Unf = range(945, 955)

### Key Recovery

In [ ]:
# Falcon_Key_Recovery
print("*"+tag+"*")
for B in Ran_KR:
    # Attack succeed when Left > Right
    # with attack costs 2^(0.292*B) [Core-SVP]
    left = numerical_approx((B/2/pi/e)^(1-n/B)*sqrt(q))
    right = numerical_approx(1.17*sqrt(3/4*B*q/2/n))
    suc = (left>right)
    print("* B="+str(B)+":: Left:", left, "   //   Right:", right, "\n** Left", ('>' if suc else '<'), "Right:: ", ("Attack cost: "+str(0.292*B) if suc else "Attack failed!"))

### Key Recovery (lattice estimator)

In [ ]:
schemes.Falcon512_SKR

In [ ]:
r = NTRU.estimate(schemes.Falcon512_SKR, red_cost_model=ADPS16("classical"))

In [ ]:
schemes.Falcon1024_SKR

In [ ]:
r = NTRU.estimate(schemes.Falcon1024_SKR, red_cost_model=ADPS16("classical"))

### Signature Forgery

In [ ]:
# Falcon_Signature_Forgery
print("*"+tag+"*")
for B in Ran_Unf:
    # Attack succeed when Left < Right
    # with attack costs 2^(0.292*B) [Core-SVP]
    left = numerical_approx((B/2/pi/e)^(n/B)*sqrt(q))
    right = numerical_approx(sqrt(betaSquared))
    suc = (left<right)
    print("* B="+str(B)+":: Left:", left, "   //   Right:", right, "\n** Left", ('<' if suc else '>'), "Right:: ", ("Attack cost: "+str(0.292*B) if suc else "Attack failed!"))

### Signature Forgery (lattice estimator)

In [ ]:
schemes.Falcon512_Unf

In [ ]:
r = SIS.estimate(schemes.Falcon512_Unf, red_cost_model=ADPS16("classical"))

In [ ]:
schemes.Falcon1024_Unf

In [ ]:
r = SIS.estimate(schemes.Falcon1024_Unf, red_cost_model=ADPS16("classical"))

## Dilithium

### Parameters

In [ ]:
n = 256
q = 8380417
d = 13

In [ ]:
# Dilithium2
tag = "Dilithium2"
k = 4
l = 4
eta = 4
tau = 39
gamma1 = 2**17
gamma2 = (q-1)/88

In [ ]:
# Dilithium3
tag = "Dilithium3"
k = 6
l = 5
eta = 2
tau = 49
gamma1 = 2**19
gamma2 = (q-1)/32

In [ ]:
# Dilithium5
tag = "Dilithium5"
k = 8
l = 7
eta = 2
tau = 60
gamma1 = 2**19
gamma2 = (q-1)/32

### Key Recovery

In [ ]:
Dilithium_KR = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.Uniform(-eta,eta),
                	Xe=NoiseDistribution.Uniform(-eta,eta),
                	m=n*l, tag=tag)

print("*"+tag+"*")
r=LWE.estimate(Dilithium_KR, red_cost_model=ADPS16("classical"))

### Signature Forgery

In [ ]:
beta = tau*eta
zeta = max(gamma1-beta, 2*gamma2 + 1 + 2**(d-1)*tau)
zetap = max(2*(gamma1-beta), 4*gamma2+2)

print("*"+tag+"*")
print("SelfTargetMSIS bound:", zeta)
print("MSIS bound:", zetap)

In [ ]:
WeakUnf = SIS.Parameters(
    n=n*k,
    q=q,
    length_bound=zeta,
    m=n*(k+l),
    norm=oo,
    tag="WeakUnf"
)

print("*"+tag+": WeakUnf*")
r = SIS.estimate(WeakUnf, red_cost_model=ADPS16("classical"))

In [ ]:
StrongUnf = SIS.Parameters(
    n=n*k,
    q=q,
    length_bound=zetap,
    m=n*(k+l),
    norm=oo,
    tag="StrongUnf"
)

print("*"+tag+": StrongUnf*")
r = SIS.estimate(StrongUnf, red_cost_model=ADPS16("classical"))

### Signature Forgery (lattice estimator)

In [ ]:
schemes.Dilithium2_MSIS_WkUnf

In [ ]:
r = SIS.estimate(schemes.Dilithium2_MSIS_WkUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium2_MSIS_WkUnf, red_cost_model=ADPS16("classical"))

In [ ]:
schemes.Dilithium2_MSIS_StrUnf 

In [ ]:
r = SIS.estimate(schemes.Dilithium2_MSIS_StrUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium2_MSIS_StrUnf, red_cost_model=ADPS16("classical"))

### Challenge Entropy

In [ ]:
# Challenge Entropy
# Challenge has a Hamming weight of tau, and each coeff is 0 or +-1
# Thus, |challenge space| = (n choose tau)*2^tau
print("*"+tag+"*")
print("Challenge Entropy:", numerical_approx(log(binomial(n, tau))/log(2)))